# Parkisons Disease Detection using Machine Learning


In [ ]:
# Importing Libraries
import requests
import pandas as pd
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB as Naive_Bayes
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.metrics import ConfusionMatrixDisplay
from xgboost import XGBClassifier
import joblib


#from IPython.display import display

# Data Collection


---


**Dataset Used :** Parkinsons Disease Dataset <br>
**Dataset Source :** UCI Machine Learning Repository <br>
**Dataset Hosting URL :** https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data <br>

In [ ]:
# URL For Data Files
url_string = 'https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data'

In [ ]:
# Downloading Content From URL & Storing Into Local File
url_content = requests.get(url_string).content
with open('data.csv', 'wb') as data_file:
  data_file.write(url_content)

In [ ]:
df = pd.read_csv('data.csv') #Read the file

In [ ]:
df

In [ ]:
df.shape

# Data Preprocessing


---
 The following steps are performed on the dataset in this section:
 + Dropping Redudant Columns
 + Checking For Duplicated Rows
 + Checking For Missing Values

In [ ]:
# Exploring Dataset Content
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.shape[0]

In [ ]:
print('Number of Features In Dataset :', df.shape[1])
print('Number of Instances In Dataset : ', df.shape[0])

The column **name**, is a *Redundant* column which is not useful for Analysis or Machine Learning, and will be dropped from the dataframe.

In [ ]:
# Dropping The Name Column
df.drop(['name'], axis=1, inplace=True)

In [ ]:
print('Number of Features In Dataset :', df.shape[1])
print('Number of Instances In Dataset : ', df.shape[0])

In [ ]:
df.info()

In [ ]:
df.describe()

It can be observed that the column **Status** is stored as *int64* datatype. However, since the column contains only two numeric values **0 & 1**, we will be changing the datatype to *uint8*, to save Memory Space. 

In [ ]:
df['status'] = df['status'].astype('uint8')

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
# Checking For Duplicate Rows In Dataset
print('Number of Duplicated Rows :',df.duplicated().sum())

As observed in the above step, the dataset does **NOT** contain any Duplicated Rows.

In [ ]:
# Checking For Missing Values In Dataset
df.isna().sum()

As seen in the above step, **No Columns** of the dataset contains any Missing Values.

# Exploratry Data Analysis

In [ ]:
df['status'].value_counts()

In [ ]:
df['status'].unique()

In [ ]:
#Balance of Data
sns.countplot(x='status',hue='status', data=df)
plt.title('Count of Status Values')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))  
sns.heatmap(df.corr(),annot=True,ax=ax)

In this correlation heatmap, we can see that many independent features are highly correlated with eachother.

In [ ]:
#Box Plot
fig,axes=plt.subplots(5,5,figsize=(15,15))
axes=axes.flatten()

for i, col in enumerate(df.columns[0:-1]):
    sns.boxplot(x='status',hue='status',y=col,data=df,orient='v',ax=axes[i])
    axes[i].set_title(col)
plt.tight_layout()
plt.show()

From the boxplot shown above it is very evident that if a patient has a lower rate of 'HNR','MDVP:Flo(Hz)','MDVP:Fhi(Hz)','MDVP:Fo(Hz)' ,then he/she is affected by parkinsons disease.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 4)
sns.pairplot(df,hue = 'status', vars = ['MDVP:Jitter(%)','MDVP:Jitter(Abs)','MDVP:RAP','MDVP:PPQ', 'Jitter:DDP'] )
plt.show()

From the above pair plot we can understand that all these fundamental frequencies are highly correlated with eachother.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 4)
sns.pairplot(df,hue = 'status', vars = ['MDVP:Shimmer','MDVP:Shimmer(dB)','Shimmer:APQ3','Shimmer:APQ5','MDVP:APQ','Shimmer:DDA'] )
plt.show()

From the above pair plot we can understand that all these measures variation in amplitude are highly correlated with eachother.



# Balancing Dataset
---
In this section, as it is observed that the Dataset Is Heavily Imbalanced, with Number of Samples of Parkinson Disease Samples being 147, and Non-Parkinson Being only 48. Hence, in this section, we make use of **SMOTE** to **Oversample** and Balance the dataset.

In [ ]:
# Exploring Imabalance In Dataset
df['status'].value_counts()

In [ ]:
df['status']

In [ ]:
# Extracting Features Into Features & Target
X = df.drop(['status'], axis=1)
y = df['status']

print('Feature (X) Shape Before Balancing :', X.shape)
print('Target (y) Shape Before Balancing :', y.shape)

In [ ]:
# Intialising SMOTE Object
sm = SMOTE(random_state=300)

In [ ]:
# Resampling Data
X, y = sm.fit_resample(X, y)

In [ ]:
print('Feature (X) Shape After Balancing :', X.shape)
print('Target (y) Shape After Balancing :', y.shape)

In [ ]:
# Scaling features between -1 and 1  for mormalization 
scaler = MinMaxScaler((-1,1))

In [ ]:
# define X_features , Y_labels
X_features = scaler.fit_transform(X)
Y_labels = y

In [ ]:
# splitting the dataset into traning and testing sets into 80 - 20
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_features, Y_labels , test_size=0.20, random_state=20)

# Machine Learning Model Training
In this section, we have trained the following Machine Learning Models:
+ Decision Tree Classifier
+ Random Forest Classifier
+ Logistic Regression
+ SVM 
+ Naive Bayes
+ KNN Classifier
+ XGBoost Classifier

## Decision Tree Classifier

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
predDT = clf.predict(X_test)

print(classification_report(y_test, predDT))

In [ ]:
n_features = X_train.shape[1]

# Adjust max_features dynamically
valid_max_features = ['sqrt', 'log2', None]
if n_features < 4:
    valid_max_features = [None]

param_grid = { 
    'max_features': valid_max_features,
    'max_depth' :range(1,10),
    'random_state':range(30,210,30),
    'criterion' :['gini', 'entropy']
}
clf = DecisionTreeClassifier()

CV_dt = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5, scoring='accuracy', error_score='raise')
CV_dt.fit(X_train, y_train)

print("Best Parameters:", CV_dt.best_params_)
print("Best Accuracy:", CV_dt.best_score_)

In [ ]:
CV_dt.best_params_

In [ ]:
dt1=DecisionTreeClassifier(random_state=120, max_features='sqrt', max_depth=6, criterion='entropy')
dt1.fit(X_train, y_train)
predDT = dt1.predict(X_test) 
print(classification_report(y_test, predDT))

In [ ]:
cm = confusion_matrix(y_test, predDT) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dt1.classes_)
disp.plot(cmap='Blues')

In [ ]:
y_pred_proba = dt1.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Dumping Decision Tree Classifier
joblib.dump(dt1, 'dt_clf.pkl')

## Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
predRF = rfc.predict(X_test)

print(classification_report(y_test, predRF))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Step 1: Calculate features and adjust max_features
n_features = X_train.shape[1]
valid_max_features = ['sqrt', 'log2', None] if n_features >= 4 else [None]

# Step 2: Define parameter grid
param_grid = {
    'n_estimators': range(100, 300, 25),
    'max_features': valid_max_features,
    'max_depth': range(1, 10),
    'criterion': ['gini', 'entropy']
}

# Step 3: Initialize and perform GridSearch
rfc = RandomForestClassifier(random_state=42)  # Random state fixed for reproducibility
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='accuracy', error_score='raise', n_jobs=-1)
CV_rfc.fit(X_train, y_train)

# Step 4: Display results
print("Best Parameters:", CV_dt.best_params_)
print("Best Accuracy:", CV_dt.best_score_)


In [ ]:
CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=200, max_features='sqrt', n_estimators= 125, max_depth=6, criterion='entropy')
rfc1.fit(X_train, y_train)
predRFC = rfc1.predict(X_test)
print(classification_report(y_test, predRFC))

In [ ]:
cm = confusion_matrix(y_test, predRFC) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rfc1.classes_)
disp.plot(cmap='Blues')

In [ ]:
y_pred_proba = rfc1.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Dumping Random Forest Classifier
joblib.dump(rfc1, 'rf_clf.pkl')

## Logistic Regression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)
predlog = logmodel.predict(X_test)

In [ ]:
print(classification_report(y_test, predlog))
print("Confusion Matrix:")
confusion_matrix(y_test, predlog)

In [ ]:
cm = confusion_matrix(y_test, predlog) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logmodel.classes_)
disp.plot(cmap='Blues')

In [ ]:
y_pred_proba = logmodel.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Dumping Logistic Regression Model
joblib.dump(logmodel, 'lg_clf.pkl')

## SVM

SVM With Linear Kernel

In [ ]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Test Set Accuracy:",metrics.accuracy_score(y_test, y_pred))

X_pred = clf.predict(X_train)
print("Train Set Accuracy:",metrics.accuracy_score(y_train, X_pred))

In [ ]:
param_grid = {'kernel':['linear','rbf','poly'],'C': [0.5, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}

grid_SVC = GridSearchCV(svm.SVC(), param_grid, scoring='f1', verbose = 3)
grid_SVC.fit(X_train, y_train)

# print best parameter after tuning 
print("\nBest Parameters: ", grid_SVC.best_params_)

# print how our model looks after hyper-parameter tuning
print("\n", grid_SVC.best_estimator_)

predSVC = grid_SVC.predict(X_test) 
  
# print classification report 
print("\n", classification_report(y_test, predSVC)) 

In [ ]:
cm = confusion_matrix(y_test, predSVC) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot(cmap='Blues')

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  predSVC)
auc = metrics.roc_auc_score(y_test, predSVC)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Dumping SVM Classifier
joblib.dump(grid_SVC, 'svm_clf.pkl')

## Naive Bayes 

In [ ]:
# Naive Bayes

gnb = Naive_Bayes()
gnb.fit(X_train, y_train)
predgnb = gnb.predict(X_test)

print(classification_report(y_test, predgnb))

In [ ]:
print("Confusion Matrix:")
confusion_matrix(y_test, predgnb)

In [ ]:
# scores -check how efficiently labels are predicted
accuracy_testing = accuracy_score(y_test, predgnb)
print("Accuracy % :",accuracy_testing*100)

In [ ]:
cm = confusion_matrix(y_test, predgnb) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gnb.classes_)
disp.plot(cmap='Blues')

In [ ]:
y_pred_proba = gnb.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Dumping Naive Bayes Classifier
joblib.dump(gnb, 'nb_clf.pkl')

## KNN Classifier


In [ ]:
import numpy as np

Ks = 10
mean_acc = []
ConfustionMx = [];
for n in range(2,Ks):
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc.append(metrics.accuracy_score(y_test, yhat))  
print('Neighbor Accuracy List')
print(mean_acc)

In [ ]:
plt.plot(range(2,Ks),mean_acc,'g')
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbours (K)')
plt.tight_layout()
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
predKNN = knn.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, predKNN) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn.classes_)
disp.plot(cmap='Blues')

In [ ]:
y_pred_proba = knn.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Dumping KNN Classifier
joblib.dump(knn, 'knn_clf.pkl')

## XGBoost Classifer
In this section, we have trained a XGBoost Classifier, for classification of Instances to be Parkinsons or Not. The following parameters of the XGBoost Classifier have been optimized in this section:
+ **Max Depth**: This value is used to determine the Maximum Depth of the Tree.
+ **ETA** : This is also known as Learning Rate.
+ **Reg_Lambda** : This is the L2 Regularization for the weights.
+ **Random State** : This is used to evaluate and determine the performance of the model based on different random states.

The *Parameter Optimization* has been performed using **GridSearchCV** with the following parameters: 
+ **Scoring Parameter**: F1 Score
+ **Cross Validation**: 3

In [ ]:
# Defining Parameter Dictionary
param_dict = {'max_depth': range(4,8), 
              'learning_rate' : [0.1, 0.2, 0.3, 0.4, 0.5],
              'reg_lambda' : [0.8, 0.9, 1, 1.1, 1.2],
              'random_state': [300, 600, 900]
}

In [ ]:
param_dict_rf = {'max_depth': range(4, 8), 'n_estimators': [10, 50, 100], 'random_state': [300, 600, 900]}
rf_clf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_dict_rf, scoring='f1', cv=3, verbose=1)
rf_clf.fit(X_train, y_train)
print("Best Score:", rf_clf.best_score_)
print("Best Parameters:", rf_clf.best_params_)


In [ ]:
import xgboost as xgb

In [ ]:
import xgboost as xgb
from sklearn.metrics import f1_score

# Convert data to DMatrix (XGBoost's native format)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_test, label=y_test)

# Define hyperparameters
params = {
    'max_depth': 6,
    'learning_rate': 0.3,
    'reg_lambda': 1.0,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'seed': 42
}

# Train model
num_boost_round = 50
model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

# Predict on validation set
y_pred = model.predict(dval)
y_pred_binary = (y_pred > 0.5).astype(int)  # Threshold at 0.5

# Evaluate F1 score
f1 = f1_score(y_test, y_pred_binary)
print("XGBoost F1 Score:", f1)


In [ ]:
# plot_confusion_matrix(xgb_clf, X_test, y_test, cmap=plt.cm.Blues) 
# plt.title('Confusion matrix for XGBoost', y=1.1)
# plt.show()
cm_xgb = confusion_matrix(y_test, y_pred_binary)

# Display confusion matrix for XGBoost
disp = ConfusionMatrixDisplay(confusion_matrix=cm_xgb, display_labels=['Class 0', 'Class 1'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix for XGBoost')
plt.show()

In [ ]:
# Dumping XGBoost Classifier
joblib.dump(model, 'xgb_clf.pkl')

# Comparision Table

In [ ]:
# Get predictions for test set
dtest = xgb.DMatrix(X_test)
y_pred = model.predict(dtest)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to 0/1



In [ ]:
from sklearn.metrics import precision_score,recall_score ,accuracy_score, f1_score, r2_score, log_loss

chart = {
        'Metric':["Accuracy", "F1-Score", "Recall", "Precision", "R2-Score"],
        'DT':[accuracy_score(y_test, predDT), f1_score(y_test, predDT), recall_score(y_test, predDT), precision_score(y_test, predDT), r2_score(y_test, predDT)],
        'RF':[accuracy_score(y_test, predRF), f1_score(y_test, predRF), recall_score(y_test, predRF), precision_score(y_test, predRF), r2_score(y_test, predRF)],
        'LR':[accuracy_score(y_test, predlog), f1_score(y_test, predlog), recall_score(y_test, predlog), precision_score(y_test, predlog), r2_score(y_test, predlog)],
        'SVM':[accuracy_score(y_test, predSVC), f1_score(y_test, predSVC), recall_score(y_test, predSVC), precision_score(y_test, predSVC), r2_score(y_test, predSVC)],
        'NB':[accuracy_score(y_test, predgnb), f1_score(y_test, predgnb), recall_score(y_test, predgnb), precision_score(y_test, predgnb), r2_score(y_test, predgnb)],
        'KNN':[accuracy_score(y_test, predKNN), f1_score(y_test, predKNN), recall_score(y_test, predKNN), precision_score(y_test, predKNN), r2_score(y_test, predKNN)],
        'XGB':[accuracy_score(y_test, y_pred_binary), f1_score(y_test, y_pred_binary), recall_score(y_test, y_pred_binary), precision_score(y_test, y_pred_binary), r2_score(y_test, y_pred_binary)]
}
chart = pd.DataFrame(chart)

In [ ]:
display(chart)